In [0]:
from pyspark.sql.types import StringType,StructField,IntegerType,FloatType,StructType

In [0]:
# Restaurant Name,City,Locality,Cuisines,Average Cost for two,Has Table booking,Has Online delivery, Rating Stars out of 5,Rating in text,Price range,Votes
custschema= StructType([
    StructField("Restaurant Name",StringType()),
    StructField("City",StringType()),
    StructField("Locality",StringType()),
    StructField("Cuisines",StringType()),
    StructField("Average Cost for two",IntegerType()),
    StructField("Has Table booking",StringType()),
    StructField("Has Online delivery",StringType()),
    StructField("Rating Stars out of 5",FloatType()),
    StructField("Rating in text",StringType()),
    StructField("Price range",IntegerType()),
    StructField("Votes",StringType())


])

In [0]:
source_df=spark.read.csv("/FileStore/tables/Swiggy_Analysis_Source_File-1.csv",header=True,schema=custschema)
source_df.printSchema()

root
 |-- Restaurant Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: integer (nullable = true)
 |-- Has Table booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |-- Rating Stars out of 5: float (nullable = true)
 |-- Rating in text: string (nullable = true)
 |-- Price range: integer (nullable = true)
 |-- Votes: string (nullable = true)



In [0]:
from pyspark.sql import functions as f

# city table

In [0]:
CITY_DIM=source_df.select(f.col("City").alias("City_Name")).distinct().withColumn("City_ID",(f.monotonically_increasing_id().cast(IntegerType()))+1)
CITY_DIM.show(5)

+---------+-------+
|City_Name|City_ID|
+---------+-------+
|Bangalore|      1|
|    Patna|      2|
|  Chennai|      3|
|  Lucknow|      4|
|   Mumbai|      5|
+---------+-------+
only showing top 5 rows



# restaurant table

In [0]:
RESTAURANT_NAME_DIM=source_df.select(f.col("Restaurant Name").alias("Restaurant_Name")).distinct().withColumn("Restaurant_ID",(f.monotonically_increasing_id().cast(IntegerType()))+1)
RESTAURANT_NAME_DIM.show(5)

+--------------------+-------------+
|     Restaurant_Name|Restaurant_ID|
+--------------------+-------------+
|            Chung Fa|            1|
|             Bar Bar|            2|
|Zolocrust - Hotel...|            3|
|            Paradise|            4|
|        Olive Bistro|            5|
+--------------------+-------------+
only showing top 5 rows



# cuisine table

In [0]:
CUISINE_DIM=source_df.select(f.col("Cuisines").alias("Cuisine_Name")).distinct().withColumn("Cuisine_ID",(f.monotonically_increasing_id().cast(IntegerType()))+1)
CUISINE_DIM.show(5)

+--------------+----------+
|  Cuisine_Name|Cuisine_ID|
+--------------+----------+
|       Mexican|         1|
|        Bakery|         2|
|      European|         3|
|Charcoal Grill|         4|
|       Biryani|         5|
+--------------+----------+
only showing top 5 rows



# locality table

In [0]:
LOCALITY_DIM=source_df.select(f.col("Locality").alias("Locality_Name")).distinct().withColumn("Locality_ID",(f.monotonically_increasing_id().cast(IntegerType()))+1)
LOCALITY_DIM.show(5)

+--------------------+-----------+
|       Locality_Name|Locality_ID|
+--------------------+-----------+
|Holiday Inn Jaipu...|          1|
|           Lal Kothi|          2|
|     Deccan Gymkhana|          3|
|International Bus...|          4|
|             Lodipur|          5|
+--------------------+-----------+
only showing top 5 rows



# table booking

In [0]:
TABLE_BOOKING_DIM=source_df.select(f.col("Has Table booking").alias("Table_Booking_avail")).distinct().withColumn("Table_Booking_id",(f.monotonically_increasing_id().cast(IntegerType()))+1)
TABLE_BOOKING_DIM.show(5)

+-------------------+----------------+
|Table_Booking_avail|Table_Booking_id|
+-------------------+----------------+
|                 No|               1|
|                Yes|               2|
+-------------------+----------------+



# delivery

In [0]:
DELIVERY_DIM=source_df.select(f.col("Has online Delivery").alias("Delivery_avail")).distinct().withColumn("Delivery_ID",(f.monotonically_increasing_id().cast(IntegerType()))+1)
DELIVERY_DIM.show(5)

+--------------+-----------+
|Delivery_avail|Delivery_ID|
+--------------+-----------+
|            No|          1|
|           Yes|          2|
+--------------+-----------+



# rating

In [0]:
RATING_DIM=source_df.select(f.col("Rating stars out of 5").alias("Rating_in_stars"),(f.col("Rating in text").alias("Rating_in_text"))).distinct().withColumn("Rating_ID",(f.monotonically_increasing_id().cast(IntegerType()))+1)
RATING_DIM.show(5)

+---------------+--------------+---------+
|Rating_in_stars|Rating_in_text|Rating_ID|
+---------------+--------------+---------+
|            3.5|          Good|        1|
|            4.9|     Excellent|        2|
|            4.3|     Very Good|        3|
|            3.9|          Good|        4|
|            4.7|     Excellent|        5|
+---------------+--------------+---------+
only showing top 5 rows



# fact table

In [0]:
source_df.show(5)

+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+---------------------+--------------+-----------+-----+
|     Restaurant Name|     City|            Locality|     Cuisines|Average Cost for two|Has Table booking|Has Online delivery|Rating Stars out of 5|Rating in text|Price range|Votes|
+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+---------------------+--------------+-----------+-----+
|    Sultans of Spice|Bangalore|BluPetal Hotel, K...| North Indian|                1300|              Yes|                Yes|                  4.1|     Very Good|          3|  314|
|The Fatty Bao - A...|Bangalore|         Indiranagar|        Asian|                2400|              Yes|                Yes|                  4.7|     Excellent|          4|  591|
|                Toit|Bangalore|         Indiranagar|      Italian|                2000|  

In [0]:
fact_table=source_df.join(CITY_DIM,CITY_DIM['City_Name']==source_df['City'],'left')\
  .join(RESTAURANT_NAME_DIM,RESTAURANT_NAME_DIM['Restaurant_Name']==source_df['Restaurant Name'],'left')\
    .join(CUISINE_DIM,CUISINE_DIM['Cuisine_Name']==source_df['Cuisines'],'left')\
      .join(LOCALITY_DIM,LOCALITY_DIM['Locality_name']==source_df['Locality'],'left')\
        .join(TABLE_BOOKING_DIM,TABLE_BOOKING_DIM['Table_Booking_avail']==source_df['Has Table Booking'],'left')\
          .join(DELIVERY_DIM,DELIVERY_DIM['Delivery_avail']==source_df['Has online Delivery'],'left')\
            .join(RATING_DIM,RATING_DIM['Rating_in_stars']==source_df['Rating stars out of 5'],'left')
fact_table.show()

+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+---------------------+--------------+-----------+-----+---------+-------+--------------------+-------------+-------------+----------+--------------------+-----------+-------------------+----------------+--------------+-----------+---------------+--------------+---------+
|     Restaurant Name|     City|            Locality|     Cuisines|Average Cost for two|Has Table booking|Has Online delivery|Rating Stars out of 5|Rating in text|Price range|Votes|City_Name|City_ID|     Restaurant_Name|Restaurant_ID| Cuisine_Name|Cuisine_ID|       Locality_Name|Locality_ID|Table_Booking_avail|Table_Booking_id|Delivery_avail|Delivery_ID|Rating_in_stars|Rating_in_text|Rating_ID|
+--------------------+---------+--------------------+-------------+--------------------+-----------------+-------------------+---------------------+--------------+-----------+-----+---------+-------+-----

In [0]:
fact_table.printSchema()

root
 |-- Restaurant Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Locality: string (nullable = true)
 |-- Cuisines: string (nullable = true)
 |-- Average Cost for two: integer (nullable = true)
 |-- Has Table booking: string (nullable = true)
 |-- Has Online delivery: string (nullable = true)
 |-- Rating Stars out of 5: float (nullable = true)
 |-- Rating in text: string (nullable = true)
 |-- Price range: integer (nullable = true)
 |-- Votes: string (nullable = true)
 |-- City_Name: string (nullable = true)
 |-- City_ID: integer (nullable = true)
 |-- Restaurant_Name: string (nullable = true)
 |-- Restaurant_ID: integer (nullable = true)
 |-- Cuisine_Name: string (nullable = true)
 |-- Cuisine_ID: integer (nullable = true)
 |-- Locality_Name: string (nullable = true)
 |-- Locality_ID: integer (nullable = true)
 |-- Table_Booking_avail: string (nullable = true)
 |-- Table_Booking_id: integer (nullable = true)
 |-- Delivery_avail: string (nullable = true)
 |--

# final fact

In [0]:
final_fact=fact_table.withColumn("FACT_ID",(f.monotonically_increasing_id()).cast(IntegerType())+1).select("FACT_ID","City_ID","Locality_ID","Restaurant_ID","Cuisine_ID",f.col("Average Cost for two").alias("Average_cost_for_two"),"Rating_ID","Delivery_ID","Table_booking_ID","Votes",f.col("Price range").alias("Price_range"))
final_fact.show(5)

+-------+-------+-----------+-------------+----------+--------------------+---------+-----------+----------------+-----+-----------+
|FACT_ID|City_ID|Locality_ID|Restaurant_ID|Cuisine_ID|Average_cost_for_two|Rating_ID|Delivery_ID|Table_booking_ID|Votes|Price_range|
+-------+-------+-----------+-------------+----------+--------------------+---------+-----------+----------------+-----+-----------+
|      1|      1|        123|          180|        18|                1300|       14|          2|               2|  314|          3|
|      2|      1|          7|          225|        23|                2400|        5|          2|               2|  591|          4|
|      3|      1|          7|           35|        14|                2000|       16|          1|               1|  270|          4|
|      4|      1|          7|           20|         3|                1300|        4|          1|               2|  365|          3|
|      5|      1|          7|           98|        16|               

# count of restaurant city wise

In [0]:
count_df=CITY_DIM.join(final_fact,"City_ID","inner").select("City_Name").groupBy("City_Name").count()
count_df.show(5)




+---------+-----+
|City_Name|count|
+---------+-----+
|Bangalore|   20|
|    Patna|   20|
|  Chennai|   20|
|  Lucknow|   21|
|   Mumbai|   20|
+---------+-----+
only showing top 5 rows



###  	City-wise Top 10 restaurant based on Avg cost for 2

In [0]:
from pyspark.sql import Window

In [0]:
my_fun=Window.partitionBy("City_Name").orderBy(f.desc("Average_cost_for_two"))

In [0]:
CRA=CITY_DIM.join(final_fact,CITY_DIM.City_ID==final_fact.City_ID)\
    .join(RESTAURANT_NAME_DIM,final_fact.Restaurant_ID==RESTAURANT_NAME_DIM.Restaurant_ID)\
        .sort("City_Name","Average_cost_for_two",ascending=False)\
            .withColumn("rank",f.row_number().over(my_fun))\
                .select("City_Name","Restaurant_Name","Average_cost_for_two")\
                    .filter(f.col("rank")<=10)
CRA.show(40)

+----------+--------------------+--------------------+
| City_Name|     Restaurant_Name|Average_cost_for_two|
+----------+--------------------+--------------------+
| Bangalore|The Fatty Bao - A...|                2400|
| Bangalore|                Toit|                2000|
| Bangalore|         Big Brewsky|                1800|
| Bangalore|    Bombay Brasserie|                1500|
| Bangalore|  Koramangala Social|                1500|
| Bangalore|          Farzi Cafe|                1500|
| Bangalore|     The Black Pearl|                1400|
| Bangalore|AB's - Absolute B...|                1400|
| Bangalore|                Hoot|                1400|
| Bangalore|    Sultans of Spice|                1300|
|Chandigarh|    Virgin Courtyard|                2200|
|Chandigarh|        TGI Friday's|                1800|
|Chandigarh|    Brooklyn Central|                1600|
|Chandigarh|    Kylin Experience|                1600|
|Chandigarh|             Chili's|                1400|
|Chandigar

### 	Top 10 restaurant based on Avg Votes

In [0]:
my_fun2=Window.partitionBy("City").orderBy(f.desc("Votes"))

In [0]:
AVG=CITY_DIM.join(final_fact,CITY_DIM.City_ID==final_fact.City_ID)\
    .join(RESTAURANT_NAME_DIM,final_fact.Restaurant_ID==RESTAURANT_NAME_DIM.Restaurant_ID)\
        .sort("Votes",ascending=False)\
            .withColumn("rank",f.monotonically_increasing_id())\
                .select("City_Name","Restaurant_Name","Votes")\
                    .filter(f.col("rank")<=10)
AVG.show(15)

+----------+--------------------+-----+
| City_Name|     Restaurant_Name|Votes|
+----------+--------------------+-----+
|   Lucknow|         Spice Caves|   98|
|    Mumbai|     The Rolling Pin|   96|
| Hyderabad|Jonathan's Kitche...|  917|
|Chandigarh|    Kylin Experience|    9|
|Chandigarh|        Super Donuts|    9|
|Chandigarh|           Taco Bell|    9|
|   Chennai|           Palmshore|    9|
|       Goa|The Fisherman's W...|    9|
|  Guwahati|       Caf� Riverrun|   88|
|   Kolkata|    Santa's Fantasea|  879|
|   Kolkata|       Flame & Grill|  879|
+----------+--------------------+-----+



### 	Top 10 restaurant based on Rating City wise

In [0]:
my_fun3=Window.partitionBy("City_Name").orderBy(f.desc("Rating_in_stars"))

In [0]:
rating_df=CITY_DIM.join(final_fact,CITY_DIM.City_ID==final_fact.City_ID)\
    .join(RESTAURANT_NAME_DIM,final_fact.Restaurant_ID==RESTAURANT_NAME_DIM.Restaurant_ID)\
        .join(RATING_DIM,'Rating_ID')\
            .sort("Rating_in_stars",ascending=False)\
                .withColumn("rank",f.row_number().over(my_fun3))\
                    .select("City_Name","Restaurant_Name","Rating_in_stars")\
                        .filter(f.col("rank")<=10)
rating_df.show(40)

+----------+--------------------+---------------+
| City_Name|     Restaurant_Name|Rating_in_stars|
+----------+--------------------+---------------+
| Bangalore|                Toit|            4.8|
| Bangalore|The Fatty Bao - A...|            4.7|
| Bangalore|  ECHOES Koramangala|            4.7|
| Bangalore|            Truffles|            4.7|
| Bangalore|              Onesta|            4.6|
| Bangalore|AB's - Absolute B...|            4.6|
| Bangalore|              Onesta|            4.6|
| Bangalore|  Koramangala Social|            4.5|
| Bangalore|         Big Brewsky|            4.5|
| Bangalore|            Flechazo|            4.4|
|Chandigarh|     Barbeque Nation|            4.5|
|Chandigarh|            Burgrill|            4.5|
|Chandigarh|    Virgin Courtyard|            4.4|
|Chandigarh|             Chili's|            4.3|
|Chandigarh|        TGI Friday's|            4.3|
|Chandigarh|    Brooklyn Central|            4.2|
|Chandigarh|           Taco Bell|            4.2|


### 	Rating based on delivery availability City wise

In [0]:
avail_city_df=CITY_DIM.join(final_fact,CITY_DIM.City_ID==final_fact.City_ID)\
    .join(DELIVERY_DIM,'Delivery_ID')\
        .join(RATING_DIM,'Rating_ID')\
                .groupBy("Delivery_avail","City_Name").avg("Rating_in_stars")\
                    .sort("City_Name","Delivery_avail")\
                        .select("City_Name","Delivery_avail",f.round(f.col("avg(Rating_in_stars)"),1).alias("Ratings"))
avail_city_df.show()

+----------+--------------+-------+
| City_Name|Delivery_avail|Ratings|
+----------+--------------+-------+
| Bangalore|            No|    4.4|
| Bangalore|           Yes|    4.4|
|Chandigarh|            No|    4.1|
|Chandigarh|           Yes|    4.0|
|   Chennai|            No|    4.4|
|   Chennai|           Yes|    4.3|
|       Goa|            No|    4.2|
|  Guwahati|            No|    4.2|
| Hyderabad|            No|    4.4|
| Hyderabad|           Yes|    4.3|
|    Jaipur|            No|    4.0|
|    Jaipur|           Yes|    4.3|
|   Kolkata|            No|    4.3|
|   Kolkata|           Yes|    4.2|
|   Lucknow|            No|    4.2|
|    Mumbai|            No|    4.0|
|    Mumbai|           Yes|    4.2|
|     Patna|            No|    3.5|
|      Pune|            No|    4.2|
|      Pune|           Yes|    4.2|
+----------+--------------+-------+
only showing top 20 rows



### 	Avg cost for 2 based on cuisine city wise

In [0]:
avg_cost_df=CITY_DIM.join(final_fact,"City_ID")\
  .join(CUISINE_DIM,"Cuisine_ID")\
    .groupBy("City_Name","Cuisine_Name").max("Average_cost_for_two")\
      .sort("City_Name","Cuisine_Name",f.col("max(Average_cost_for_two)").alias("Average_cost_for_two"))
avg_cost_df.show(40)

+----------+-------------+-------------------------+
| City_Name| Cuisine_Name|max(Average_cost_for_two)|
+----------+-------------+-------------------------+
| Bangalore|     American|                      800|
| Bangalore|        Asian|                     2400|
| Bangalore|       Bakery|                      800|
| Bangalore|  Continental|                     1500|
| Bangalore|     European|                     1400|
| Bangalore|    Fast Food|                     1800|
| Bangalore|      Italian|                     2000|
| Bangalore|Modern Indian|                     1500|
| Bangalore| North Indian|                     1400|
|Chandigarh|     American|                     1800|
|Chandigarh|       Bakery|                      650|
|Chandigarh|         Cafe|                      650|
|Chandigarh|  Continental|                     1400|
|Chandigarh|     Desserts|                      600|
|Chandigarh|    Fast Food|                      500|
|Chandigarh|      Italian|                    